In [1]:
import os, glob, random
import pandas as pd
import SimpleITK as sitk
import numpy as np
from scripts_img_process.kidney_seg_to_length import get_lateral_length_from_mask, get_image_path_by_id


In [12]:
df_init = pd.DataFrame()

manual_seg_dir =  'nnUNet_raw_data_base/nnUNet_raw_data/label_in_nii_train526_kito/'
auto_seg_dir = 'nnUNet_raw_data_base/nnUNet_raw_data/label_in_nii_train526_kito/'
csv_write_path= 'nnUNet_raw_data_base/nnUNet_raw_data/kidney_length.csv'


for idx, d in sorted(enumerate(os.listdir(manual_seg_dir))):
    if '.nii.gz' in d :
        print(d,'processing')
        indices = [i for i, c in enumerate(d) if c == '_']
        pid = d[:indices[1]]

        auto_seg_path = get_image_path_by_id(pid, auto_seg_dir)
        manual_seg_path = get_image_path_by_id(pid, manual_seg_dir)

        if os.path.exists(auto_seg_path) and os.path.exists(manual_seg_path)  :
            manual_seg_sitk = sitk.ReadImage(manual_seg_path)
            mskCube  = sitk.GetArrayFromImage(manual_seg_sitk)
            auto_seg_sitk = sitk.ReadImage(auto_seg_path)
            prdCube = sitk.GetArrayFromImage(auto_seg_sitk)
            
            mskCube_right, mskCube_left = mskCube.copy(), mskCube.copy()
            prdCube_right, prdCube_left = prdCube.copy(), prdCube.copy()
            
            mskCube_right[mskCube == 2] = 0  # get right kidney  (class 1) 
            prdCube_right[prdCube == 2] = 0  # get right kidney  (class 1) 
            mskCube_left[mskCube == 1] = 0  #  get left kidney (class 2)
            prdCube_left[prdCube == 1] = 0  # get left kidney  (class 1)  
            
            volume_right_gd = np.sum(mskCube_right) *1.0 *1.0 *5.0 /1000
            volume_right_ai = np.sum(prdCube_right)   *1.0 *1.0 *5.0 /1000         
            volume_left_gd = np.sum(mskCube_left)*1.0 *1.0 *5.0 /1000 / 2
            volume_left_ai = np.sum(prdCube_left)*1.0 *1.0 *5.0 /1000 / 2
            
            length_right_gd, len_rgd_cord0, len_rgd_cord1 = get_lateral_length_from_mask(mskCube_right) 
            length_right_ai, len_rai_cord0, len_rai_cord1 = get_lateral_length_from_mask(prdCube_right)             
            length_left_gd, len_lgd_cord0, len_lgd_cord1 = get_lateral_length_from_mask(mskCube_left)            
            length_left_ai, len_lai_cord0, len_lai_cord1 = get_lateral_length_from_mask(prdCube_left)
           

            df = pd.DataFrame({'CT_id':  pid,
                               
                        'volume_right_gd':round(volume_right_gd,5), 'volume_right_ai':round(volume_right_ai,5),
                        'volume_left_gd':round(volume_left_gd,5),'volume_left_ai':round(volume_left_ai,5),  
                               
                        'length_right_gd':round(length_right_gd,5), 'length_right_ai':round(length_right_ai,5), 
                        'length_left_gd':round(length_left_gd,5),'length_left_ai':round(length_left_ai,5),

                        'len_rgd_cord0': str(len_rgd_cord0),  'len_rai_cord0': str(len_rai_cord1),                               
                        'len_rgd_cord1': str(len_rgd_cord1) ,  'len_rai_cord1': str(len_rai_cord1),  
                        'len_lgd_cord0': str(len_lgd_cord0),  'len_lai_cord0': str(len_lai_cord0),
                        'len_lgd_cord1': str(len_lgd_cord1),  'len_lai_cord1': str(len_lai_cord1)
                            },index=[0])
            df_init = df_init.append(df)
            df_init.to_csv(csv_write_path )
            print(idx,'th',pid,'\n\n')


Colon_001_kidney_2class_label_seg_kito.nii.gz processing
94.56214887575261
94.56214887575261
94.7100839404126
94.7100839404126
0 th Colon_001 


